# **<span style="font-family: 'Palatino Linotype', serif;">Ahhh!!! Meus vizinhos não me deixam em paz!</span>**

*<span style="font-family: 'Angilla Tattoo'">Nós do Sepulcro de Delfos ficamos profundamente sensibilizados com a discussão se salmão é um peixe ou uma cor. Sentimos-nos inspirados a disseminar mais discórdia e confusão pelo reino e lenvantamos a grande dúvida: "Laranja é cor ou fruta?". A partir disso, foi feito um levantamento com tal objeto de estudo e discorremos acerca das nossas conclusões. <br> <br> Nossos algoritmos são oráculos, nossos dados são ossos ancestrais. <br> Sepulcro de Delfos </span>*

---

**Modelos 2 - Otimizando KNN**
==========================================================

**Autores:** Sepulcro de Delfos
* Ana Luz 
* Caio Ruas
* Caio Matheus
* Giovana Martins

## Introdução

Nesse caderno, nosso objetivo é estudar um pouco mais o modelo `KNN (k-Nearest Neighbors)` e sua aplicação em um conjunto de dados. Para isso, escolhemos um conjunto de dados com 3 atributos numéricos e 1 target numérico e testaremos o que acontece com a **performance** do modelo conforme modificamos o **número de vizinhos** e a **normalização dos dados**.

Dessa forma, utilizamos um conjunto de dados importado do site `Kaggle`, denominado *Orange Quality Analysis Dataset |🍊* [1] e testaremos a previsão de um target a partir do método dos k-NN vizinhos, normalizando ou não os dados para maior entendimento.

## Preparando o Ambiente

Primeiramente, vamos importar as bibliotecas necessárias para o desenvolvimento deste notebook.

In [1]:
import pandas as pd
import numpy as np

import plotly.express as px

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Incorporando os Dados

Agora, vamos importar o conjunto de dados e visualizar características para entendermos melhor o que estamos lidando.

In [2]:
df = pd.read_csv('databases_quests\Orange Quality Data.csv')
df.dtypes #busca por dados numéricos para analisar

Size (cm)             float64
Weight (g)              int64
Brix (Sweetness)      float64
pH (Acidity)          float64
Softness (1-5)        float64
HarvestTime (days)      int64
Ripeness (1-5)        float64
Color                  object
Variety                object
Blemishes (Y/N)        object
Quality (1-5)         float64
dtype: object

In [3]:
df

,Size (cm),Weight (g),Brix (Sweetness),pH (Acidity),Softness (1-5),HarvestTime (days),Ripeness (1-5),Color,Variety,Blemishes (Y/N),Quality (1-5)
0,7.5,180,12.0,3.2,2.0,10,4.0,Orange,Valencia,N,4.0
1,8.2,220,10.5,3.4,3.0,14,4.5,Deep Orange,Navel,N,4.5
2,6.8,150,14.0,3.0,1.0,7,5.0,Light Orange,Cara Cara,N,5.0
3,9.0,250,8.5,3.8,4.0,21,3.5,Orange-Red,Blood Orange,N,3.5
4,8.5,210,11.5,3.3,2.5,12,5.0,Orange,Hamlin,Y (Minor),4.5
...,...,...,...,...,...,...,...,...,...,...,...
236,8.0,194,10.9,3.6,5.0,13,1.0,Orange-Red,Tangerine,Y (Scars),5.0
237,7.4,275,8.5,3.5,5.0,20,5.0,Light Orange,Minneola (Hybrid),N,4.0
238,7.5,196,15.7,3.0,3.0,13,3.0,Deep Orange,Temple,Y (Minor Insect Damage),5.0
239,7.2,251,9.8,4.3,3.0,23,1.0,Light Orange,Moro (Blood),Y (Minor Insect Damage),3.0


Observado os dados e suas características, decidimos escolher os atributos *Size*, *Weight* (Tamanho e peso, respectivamente) e *pH* para medir o target *Sweetness* (doçura da laranja), pois eram os dados que atendiam ao critério de serem numéricos.

A seguir, podemos plotar um gráfico pra ter uma noção de como os dados se distribuem de acordo com nossos atributos.

In [4]:
fig = px.scatter_3d(df, x="Size (cm)", y="Weight (g)", z="pH (Acidity)", color="Brix (Sweetness)")
fig.show()

É possível perceber uma linha de tendência no gráfico acima, o que pode ser um indicativo de que podemos utilizar um modelo de regressão para prever o target *Sweetness*. Então, vamos lá!

## Trabalhando com o Modelo

Agora sim, vamos trabalhar com o modelo `KNN` e testar a performance do modelo conforme modificamos o número de vizinhos e a normalização dos dados.

1. Selecionando as variáveis de entrada e saída.

In [5]:
FEATURES = ["Size (cm)", "Weight (g)", "pH (Acidity)"]
TARGET = ["Brix (Sweetness)"]

df = df.dropna()

X = df.reindex(FEATURES, axis=1)
y = df.reindex(TARGET, axis=1)

X = X.sample(frac=0.9, random_state=1)
y = y.sample(frac=0.9, random_state=1)

X = X.values
y = y.values.ravel()

normalizador = MinMaxScaler()
normalizador.fit(X)
X_normalizado = normalizador.transform(X)

2. Definindo as funções responsáveis por nosso modelo.

In [6]:
def calcula_distancia(coordenada_1, coordenada_2):

    distancia = 0
    for c1, c2 in zip(coordenada_1, coordenada_2):
        distancia += (c1 - c2) ** 2
    distancia = distancia ** 0.5

    return distancia

def treinar_knn(modelo, X, y, num_vizinhos):

    modelo["atributos"] = X
    modelo["target"] = y
    modelo["num_vizinhos"] = num_vizinhos

def previsao_knn(modelo, X):

    distancias = []
    for penguin in modelo["atributos"]:
        distancia_calculada = calcula_distancia(penguin, X)
        distancias.append(distancia_calculada)
    indices = np.argsort(distancias)[:modelo["num_vizinhos"]]
    targets = modelo["target"][indices]
    y_previsto = np.mean(targets)

    return y_previsto

Agora que temos tudo pronto, vamos testar o modelo com diferentes números de vizinhos e com os dados normalizados e não normalizados.

- **Dados não normalizados**

In [7]:
NUM_VIZINHOS = list(range(1, 21))
NUM_VIZINHOS

x_novo = [8, 100, 3.8]
lista_previsões = []

for nv in NUM_VIZINHOS:
    modelo = {}
    treinar_knn(modelo, X, y, nv)
    y_previsto = previsao_knn(modelo, x_novo)
    lista_previsões.append(y_previsto)
    print(f"Com {nv} vizinhos, a previsão é {y_previsto}")

Com 1 vizinhos, a previsão é 14.5
Com 2 vizinhos, a previsão é 14.25
Com 3 vizinhos, a previsão é 12.533333333333333
Com 4 vizinhos, a previsão é 11.55
Com 5 vizinhos, a previsão é 12.24
Com 6 vizinhos, a previsão é 11.866666666666667
Com 7 vizinhos, a previsão é 12.114285714285714
Com 8 vizinhos, a previsão é 12.350000000000001
Com 9 vizinhos, a previsão é 12.533333333333335
Com 10 vizinhos, a previsão é 12.650000000000002
Com 11 vizinhos, a previsão é 12.236363636363638
Com 12 vizinhos, a previsão é 12.258333333333335
Com 13 vizinhos, a previsão é 12.446153846153846
Com 14 vizinhos, a previsão é 12.592857142857143
Com 15 vizinhos, a previsão é 12.686666666666667
Com 16 vizinhos, a previsão é 12.80625
Com 17 vizinhos, a previsão é 12.447058823529412
Com 18 vizinhos, a previsão é 12.505555555555555
Com 19 vizinhos, a previsão é 12.452631578947368
Com 20 vizinhos, a previsão é 12.285


In [8]:
fig = px.line(x=NUM_VIZINHOS, y=lista_previsões, title="Doçura das laranjas sem normalização", labels={"x": "Número de vizinhos", "y": "Doçura"})
fig.update_traces(line_color="orange")
fig.show()

In [9]:
X_teste = df.drop(pd.DataFrame(X).index)
y_teste = df.drop(pd.DataFrame(y).index)

X_teste = X_teste.reindex(FEATURES, axis=1)
y_teste = y_teste.reindex(TARGET, axis=1)

X_teste = X_teste.values
y_teste = y_teste.values.ravel()

X_teste_normalizado = normalizador.transform(X_teste)

lista_rmse = []
for nv in NUM_VIZINHOS:
    modelo = {}
    treinar_knn(modelo, X, y, nv)
    previsoes = []
    for x_novo in X_teste:
        y_previsto = previsao_knn(modelo, x_novo)
        previsoes.append(y_previsto)
    rmse = mean_squared_error(y_teste, previsoes) ** 0.5
    lista_rmse.append(rmse)
    print(f"Com {nv} vizinhos, o RMSE é {rmse}")

Com 1 vizinhos, o RMSE é 0.285773803324704
Com 2 vizinhos, o RMSE é 2.1684047515781426
Com 3 vizinhos, o RMSE é 2.580572000904422
Com 4 vizinhos, o RMSE é 2.8094659560730277
Com 5 vizinhos, o RMSE é 2.8799942129571483
Com 6 vizinhos, o RMSE é 2.7863485829892007
Com 7 vizinhos, o RMSE é 2.608740541120246
Com 8 vizinhos, o RMSE é 2.6013755876298728
Com 9 vizinhos, o RMSE é 2.661650374495002
Com 10 vizinhos, o RMSE é 2.63147534664492
Com 11 vizinhos, o RMSE é 2.7665585876197856
Com 12 vizinhos, o RMSE é 2.795892841724698
Com 13 vizinhos, o RMSE é 2.8459164835187467
Com 14 vizinhos, o RMSE é 2.806031298766973
Com 15 vizinhos, o RMSE é 2.7826150104346623
Com 16 vizinhos, o RMSE é 2.8045443034475315
Com 17 vizinhos, o RMSE é 2.8373728751710856
Com 18 vizinhos, o RMSE é 2.8359771162165086
Com 19 vizinhos, o RMSE é 2.8649057004039125
Com 20 vizinhos, o RMSE é 2.867536144788181


In [10]:
fig = px.line(x=NUM_VIZINHOS, y=lista_rmse, title="RMSE das laranjas sem normalização", labels={"x": "Número de vizinhos", "y": "RMSE"})
fig.update_traces(line_color="orange")
fig.show()

- **Dados normalizados**

In [11]:
lista_previsões_n = []

for nv in NUM_VIZINHOS:
    modelo = {}
    treinar_knn(modelo, X_normalizado, y, nv)
    y_previsto_n = previsao_knn(modelo, x_novo)
    lista_previsões_n.append(y_previsto_n)
    print(f"Com {nv} vizinhos, a previsão é {y_previsto_n}")

Com 1 vizinhos, a previsão é 8.1
Com 2 vizinhos, a previsão é 10.399999999999999
Com 3 vizinhos, a previsão é 10.633333333333333
Com 4 vizinhos, a previsão é 11.5
Com 5 vizinhos, a previsão é 11.74
Com 6 vizinhos, a previsão é 11.816666666666668
Com 7 vizinhos, a previsão é 10.914285714285715
Com 8 vizinhos, a previsão é 10.3375
Com 9 vizinhos, a previsão é 10.011111111111113
Com 10 vizinhos, a previsão é 10.14
Com 11 vizinhos, a previsão é 10.545454545454545
Com 12 vizinhos, a previsão é 10.85
Com 13 vizinhos, a previsão é 10.776923076923076
Com 14 vizinhos, a previsão é 10.478571428571428
Com 15 vizinhos, a previsão é 10.24
Com 16 vizinhos, a previsão é 10.55
Com 17 vizinhos, a previsão é 10.311764705882354
Com 18 vizinhos, a previsão é 10.211111111111112
Com 19 vizinhos, a previsão é 9.989473684210527
Com 20 vizinhos, a previsão é 9.815000000000001


In [12]:
fig = px.line(x=NUM_VIZINHOS, y=lista_previsões_n, title="Doçura das laranjas para dados normalizados", labels={"x": "Número de vizinhos", "y": "Doçura"})
fig.update_traces(line_color="orange")
fig.show()


In [13]:
lista_rmse_n = []

for nv in NUM_VIZINHOS:
    modelo = {}
    treinar_knn(modelo, X_normalizado, y, nv)
    previsoes = []
    for x_novo in X_teste_normalizado:
        y_previsto = previsao_knn(modelo, x_novo)
        previsoes.append(y_previsto)
    rmse = mean_squared_error(y_teste, previsoes) ** 0.5
    lista_rmse_n.append(rmse)
    print(f"Com {nv} vizinhos, o RMSE é {rmse}")

Com 1 vizinhos, o RMSE é 0.6327848502189876
Com 2 vizinhos, o RMSE é 2.258848342260572
Com 3 vizinhos, o RMSE é 2.5665404009346178
Com 4 vizinhos, o RMSE é 2.619990259205811
Com 5 vizinhos, o RMSE é 2.713404994958671
Com 6 vizinhos, o RMSE é 2.7201132703811624
Com 7 vizinhos, o RMSE é 2.7767897335754097
Com 8 vizinhos, o RMSE é 2.778911704321795
Com 9 vizinhos, o RMSE é 2.9158826106647284
Com 10 vizinhos, o RMSE é 2.933562254324936
Com 11 vizinhos, o RMSE é 2.970551631809348
Com 12 vizinhos, o RMSE é 2.9125491769811087
Com 13 vizinhos, o RMSE é 2.9472796101815173
Com 14 vizinhos, o RMSE é 2.905135246304331
Com 15 vizinhos, o RMSE é 2.900725962773993
Com 16 vizinhos, o RMSE é 2.838382927518619
Com 17 vizinhos, o RMSE é 2.857404533658731
Com 18 vizinhos, o RMSE é 2.889435489599944
Com 19 vizinhos, o RMSE é 2.893087249280306
Com 20 vizinhos, o RMSE é 2.8467450345379834


In [14]:
fig = px.line(x=NUM_VIZINHOS, y=lista_rmse_n, title="RMSE das laranjas para dados normalizados", labels={"x": "Número de vizinhos", "y": "RMSE"})
fig.update_traces(line_color="orange")
fig.show()


## Discussão

Primeiramente, percebemos que podemos ter duas interpretações a depender se utilizamos dados normalizados ou não, já que quando normalizamos os dados, o peso que cada atributo contribui para o cálculo da distância se torna o mesmo, visto que todos os dados estarão compreendidos entre 0 e 1 (Isso ocorre pois usamos o normalizador por mínimo e máximo). Olhando para os gráficos observamos que nos dois casos quanto mais vizinhos são analisados mais o valor da previsão se estabiliza, ou seja a previsão se torna mais e mais constante. Contudo, é curioso perceber que, para dados normalizados, temos um gráfico crescente e para os dados não normalizados temos uma curva decrescente, e que se estabilizam em valores diferentes.  

Agora, como principal estudo, é muito importante notarmos que conforme o número de vizinhos cresce o **RMSE também cresce**, o que é um indicativo de que o modelo está se tornando **menos preciso**. Essa tendência sugere que o modelo pode estar sofrendo de **overfitting**. Com poucos vizinhos, o modelo se ajusta muito aos dados de treinamento. À medida que **aumentamos o número de vizinhos**, o modelo começa a considerar pontos mais distantes, o que pode incluir pontos de outras classes ou **outliers**, capturando **ruído** ou **variações aleatórias**, levando a **previsões menos precisas**.

## Conclusão

Concluímos, portanto, que a previsão pelos k-NN vizinhos é alterada a partir da normalização dos dados e da diferente quantidade adotada de vizinhos, como pôde ser visto nos gráficos plotados. 


Caso o valor dos nossos atributos exerçam a mesma influência no cálculo do valor do nosso target, utilizaremos os dados normalizados. Entretanto, caso algum dos nossos atributos exerça uma maior influência sobre o target, o peso é parte essencial, e os dados utilizados são os não-normalizados

*<span style="font-family: 'Angilla Tattoo'">Os dados normalizados são mais vantajosos em alguns casos, sendo utilizados se você não possui o intuito de dar um "peso" aos dados. Como não há a certeza sobre **laranja ser uma fruta ou cor** (não sabemos se os dados precisam de um peso ou não), ambos foram plotados e comparados.</span>*

## Referências

1. **Orange Quality Analysis Dataset| 🍊**. Disponível em: <https://www.kaggle.com/datasets/shruthiiiee/orange-quality>. 

2. **1.6. Nearest Neighbors — scikit-learn 0.21.3 documentation**. Disponível em: <https://scikit-learn.org/stable/modules/neighbors.html>. 